In [1]:
from dataloader import load_data_one_hot_encoded
from trainer import train_model, evaluate_model
from snn import SNNModelSimple
import torch
import matplotlib.pyplot as plt
import pickle

In [ ]:
root_folder = "./tactile_dataset/"
file_name = "final_merged_df_sw500.csv"

train_losses_beta = {}
train_accuracies_beta = {}
val_losses_beta = {}
val_accuracies_beta = {}
test_accuracies_beta = {}
for beta in [0.6, 0.7, 0.8, 0.9, 0.99, 0.999, 1]:
    print(f"Training for beta={beta}")
    train_loader, val_loader, test_loader, num_outputs, num_features = load_data_one_hot_encoded(root_folder, file_name, num_steps=100, batch_size=128)

    model = SNNModelSimple(num_features, num_outputs, beta=beta)
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_losses, train_accuracies, val_losses, val_accuracies = train_model(model, train_loader, val_loader, optimizer, criterion, num_epochs=10)
    train_losses_beta[beta] = train_losses
    train_accuracies_beta[beta] = train_accuracies
    val_losses_beta[beta] = val_losses
    val_accuracies_beta[beta] = val_accuracies

    test_accuracies_beta[beta] = evaluate_model(model, test_loader, encoding="one-hot")

out_file_name = 'exp2_mse_beta_06_1_bs_128_ep_10_T_100.pkl' 
with open(out_file_name, 'wb') as f:
    pickle.dump({"train_losses": train_losses_beta, "train_accuracies": train_accuracies_beta, "val_losses": val_losses_beta, "val_accuracies": val_accuracies_beta, "test_accuracies": test_accuracies_beta}, f)
